# Project 8 -- [Karim] [El-Sharkawy]

**TA Help:** No help :)

## Question 1

**A**

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import time

# Create the Session
spark = SparkSession.builder\
    .master("local")\
    .appName("PySpark Tutorial")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 03:25:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/04 03:25:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Generate a random dataframe with 10^7 rows
data = {'col1':np.random.randint(1,10,10000000),'col2':np.random.randint(1,10,10000000)}
df = pd.DataFrame(data)

# Creating our pandas dataframe
pandasDF = df.copy()

startPandas = time.time()
pandasDF['col2'] = pandasDF['col2'].map(lambda x: x - (x*10/100))
endPandas = time.time()
print(endPandas - startPandas)

1.703458309173584


In [3]:
# Creating our spark dataframe
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled","true")
sparkDF = spark.createDataFrame(df)

/usr/local/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/usr/local/lib/python3.11/site-packages/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(series.dtype):


In [4]:
startSpark = time.time()
sparkRDD = sparkDF.rdd.map(lambda x: [x[0],x[1] - (x[1]*10/100)])
endSpark = time.time()
print(endSpark - startSpark)

0.5703299045562744


**B**

This lab is introducing us and getting us used to PySpark. Spark is at least 20 times faster than pandas with processing large amounts of data (I'm using 2 cores). I'm assuming this speed difference isn't as large when we're using small data sets, but this is good to know because now I'll just use Spark instead of PANDAS

## Question 2

In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# creating a spark application named TDM_S that uses 2 gigabytes of the spark driver program
sp = SparkSession.builder.appName('TDM_S').config("spark.driver.memory", "2g").getOrCreate()
#.getOrCreate() actually creates the spark sessions

24/03/04 03:25:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
myDF = sp.read.parquet("/anvil/projects/tdm/data/whin/weather.parquet")
myDF.show(5) #Showing the first 5 rows of the dataframe

24/03/04 03:25:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------+---------+---------------+--------------------+-----------+----------------+---------------+--------+---------------+--------------------+----+---------------------+--------------+----------------------+-------------------+---------------------------+--------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+
|station_id|latitude|longitude|           name|    observation_time|temperature|temperature_high|temperature_low|humidity|solar_radiation|solar_radiation_high|rain|rain_inches_last_hour|wind_speed_mph|wind_direction_degrees|wind_gust_speed_mph|wind_gust_direction_degrees|pressure|soil_temp_1|soil_temp_2|soil_temp_3|soil_temp_4|soil_moist_1|soil_moist_2|soil_moist_3|soil_moist_4|
+----------+--------+---------+---------------+--------------------+-----------+----------------+---------------+--------+---------------+--------------------+----+---------------------+--------------+----------------------+------------

For this problem, we're using pyspark to create a dataframe and read in weather data from WHIN. We're then showing the first 5 rows of that dataframe. It's important to note that the first question being run might affect the output, however I haven't seen a difference between mine and the output in Dr. Ward's video

The WHIN dataset contains specific meteorological data from multiple stations around the US. Each station has a station ID and contain data on the temperature, dew point, rainfall, and mnay other things :)

## Question 3

In [7]:
#A
myDF.printSchema() #Listing column names and data types of the WHIN dataframe

root
 |-- station_id: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- observation_time: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- temperature_high: double (nullable = true)
 |-- temperature_low: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- solar_radiation: double (nullable = true)
 |-- solar_radiation_high: double (nullable = true)
 |-- rain: double (nullable = true)
 |-- rain_inches_last_hour: double (nullable = true)
 |-- wind_speed_mph: double (nullable = true)
 |-- wind_direction_degrees: double (nullable = true)
 |-- wind_gust_speed_mph: double (nullable = true)
 |-- wind_gust_direction_degrees: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- soil_temp_1: double (nullable = true)
 |-- soil_temp_2: double (nullable = true)
 |-- soil_temp_3: double (nullable = true)
 |-- soil_temp_4: double (nullable = true)
 |

In [8]:
#B
# couting and print the rows
myDF.count()

7494263

In [9]:
#C
myDF.select('station_id').distinct().count() # counting the number of stations

152

For part B, the .count() function is simply counting the number of rows, not actually displaying how many unique values there are. Part c is printing how many unique values of 'station_id' there are. In other words, how may stations have data provided in part A

## Question 4

In [10]:
#A
myDF.createOrReplaceTempView("weather")

In [11]:
#B
# Get the total number of records for each station
myresults = sp.sql("SELECT station_id, COUNT(*) AS Total_Records FROM weather GROUP BY station_id")
myresults.show()

+----------+-------------+
|station_id|Total_Records|
+----------+-------------+
|       191|         7914|
|        65|        69664|
|        54|        57177|
|       167|        27498|
|       155|        43028|
|       112|        68669|
|       113|        71729|
|       130|        44085|
|        77|        32312|
|       184|        13509|
|       188|         9921|
|       126|        69271|
|       149|        44095|
|        94|        29023|
|        50|        65900|
|       190|         7735|
|       110|        71622|
|       136|        71682|
|        57|        71701|
|       144|        45530|
+----------+-------------+
only showing top 20 rows



In [12]:
#C
myresults = sp.sql("SELECT station_id, MAX(wind_speed_mph) AS Max_wind_speed FROM weather GROUP BY station_id")
myresults.show()

+----------+--------------+
|station_id|Max_wind_speed|
+----------+--------------+
|       191|          18.0|
|        65|          37.0|
|        54|          32.0|
|       167|          36.0|
|       155|          32.0|
|       112|          30.0|
|       113|          26.0|
|       130|          33.0|
|        77|           5.0|
|       184|          31.0|
|       188|          23.0|
|       126|          30.0|
|       149|          27.0|
|        94|          32.0|
|        50|          12.0|
|       190|          24.0|
|       110|          32.0|
|       136|          36.0|
|        57|          27.0|
|       144|          35.0|
+----------+--------------+
only showing top 20 rows



In [13]:
#D
myresults = sp.sql("SELECT station_id, AVG(temperature) AS avg_temperature FROM weather GROUP BY station_id")
myresults.show()

+----------+------------------+
|station_id|   avg_temperature|
+----------+------------------+
|       191| 69.37035633055345|
|        65| 52.44688029402475|
|        54|  53.7297479420121|
|       167|47.787132673843466|
|       155| 55.48980960364028|
|       112| 52.58802370793225|
|       113| 52.91307500313733|
|       130| 55.45146875354429|
|        77| 51.16190947928921|
|       184| 60.51306536383152|
|       188| 65.22620967741935|
|       126| 52.58266807177607|
|       149| 55.31502437918131|
|        94|49.612962133480345|
|        50| 51.33506828528073|
|       190| 69.67356173238527|
|       110|  52.8127391025104|
|       136| 52.40440017299348|
|        57| 52.13385540991387|
|       144| 55.07742148034263|
+----------+------------------+
only showing top 20 rows



We're creating SQL queries for parts B,C, and D. Basically, we're setting them up as tables so that we could see the data. For part B, we're seeing the stations' ID and how many records they keep. For part C, we're getting the max wind speed recorded for each station. For part D, we're getting the average temperature for each station.

For all parts, only the top 20 stations show up, but this can be changed. There are 152 stations

## Question 5

In [14]:
myresults = sp.sql("SELECT station_id, AVG(solar_radiation) AS avg_solar_radiation FROM weather GROUP BY station_id")
myresults.show()

+----------+-------------------+
|station_id|avg_solar_radiation|
+----------+-------------------+
|       191| 219.90055597675007|
|        65|               NULL|
|        54|               NULL|
|       167| 170.86899912714577|
|       155|  177.7247611979603|
|       112|               NULL|
|       113|               NULL|
|       130|               NULL|
|        77|               NULL|
|       184|  213.4608779332297|
|       188| 211.31431451612903|
|       126|               NULL|
|       149| 162.22487810409342|
|        94|               NULL|
|        50|  174.6398937784522|
|       190|  227.1566903684551|
|       110|               NULL|
|       136|               NULL|
|        57| 165.59751158534212|
|       144| 179.55416681718023|
+----------+-------------------+
only showing top 20 rows



This finds the average solar radiation in every station in watts per square meter (W/M^2). This could tell us a lot of things. First, we could hypothesis about the cloud coverage. the more clouds, the less solar radiation because the clouds are reflecting the light. Second, perhaps this is at a higher elevation, so it gets more sunlight. This is why I look at average pressure later

Just from looking at this table, I can hypothesis that stations 190 and 191 are either higher in elevation, get less cloud coverage, which could also tell us about their location without it being told to us, or it's a mix of both :)

In [15]:
myresults = sp.sql("SELECT station_id, AVG(pressure) AS avg_pressure FROM weather GROUP BY station_id")
myresults.show() # in inches Hg

+----------+------------------+
|station_id|      avg_pressure|
+----------+------------------+
|       191|30.066801111953573|
|        65|  30.0257834175469|
|        54| 30.24833401542529|
|       167| 29.19178063859176|
|       155|29.369080087385147|
|       112|30.125069594722362|
|       113| 29.89896177278859|
|       130|29.890234047861878|
|        77|30.007343742262762|
|       184|29.912333111259198|
|       188|29.983693447580652|
|       126|30.039251707063215|
|       149|29.323806281891507|
|        94|30.003139923508776|
|        50| 29.38559667678274|
|       190| 30.08626800258568|
|       110|30.045749364720386|
|       136|29.988627200691703|
|        57|30.303463675541455|
|       144|29.303840918075966|
+----------+------------------+
only showing top 20 rows



This finds the average pressure for every station in inches of mercury (in Hg). Looking at the avergae pressure is a good way to tell the height of the stations. Higher pressure means lower elevation. Since many of the stations have similar average pressure, this tells me that stations 190 and 191 have less cloud coverage than the other stations!

## Pledge

By submitting this work I hereby pledge that this is my own, personal work. I've acknowledged in the designated place at the top of this file all sources that I used to complete said work, including but not limited to: online resources, books, and electronic communications. I've noted all collaboration with fellow students and/or TA's. I did not copy or plagiarize another's work.

> As a Boilermaker pursuing academic excellence, I pledge to be honest and true in all that I do. Accountable together – We are Purdue.